In [1]:
import tensorflow as tf
import numpy as np
import glob
import sys
import scipy.misc
from scipy import misc 

In [2]:
## HELPERS

# Functions to load data, DO NOT change these
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)
        
def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [3]:

# Load the data
data_root_path = '/home/ec2-user/Neural_Network/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path)
X_train = X_train.astype(np.float32)
# this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs]
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1)
y_train = np.delete(y_train,indexs)



{'automobile': 1, 'truck': 9, 'frog': 6, 'cat': 3, 'horse': 7, 'bird': 2, 'ship': 8, 'dog': 5, 'airplane': 0, 'deer': 4}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [4]:
def cnn_model(features, labels, mode):
    LEARNING_RATE = 0.001
    DROPOUT = 0.3
    
    with tf.device('/gpu:0'):
        input_layer = tf.reshape(features['x'], [-1, 32, 32, 3])
        conv1 = tf.layers.conv2d(inputs= input_layer, filters = 64, kernel_size = 3, padding= "same", activation = tf.nn.relu)
        pool1=  tf.layers.max_pooling2d(inputs = conv1, pool_size = 2, strides = 2) # cuts size in 2
        conv2 = tf.layers.conv2d(inputs = pool1, filters = 128, kernel_size = 3, padding= "same", activation = tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = 2, strides = 2) # cuts size in 2

        conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv3')
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2, padding='SAME', name='pool3')
        pool3_flat = tf.reshape(pool3, [-1, 4*4*128], name='pool3_flat')
        dropout1 = tf.layers.dropout(pool3_flat, rate=DROPOUT, training=(mode == tf.estimator.ModeKeys.TRAIN), name='dropout1')
        dense1 = tf.layers.dense(dropout1, units = 8*128, activation = tf.nn.relu, name='dense1')
        dropout2 = tf.layers.dropout(dense1, rate=DROPOUT, training=(mode == tf.estimator.ModeKeys.TRAIN), name='dropout2')
        dense2 = tf.layers.dense(dropout2, units = 128, activation = tf.nn.relu, name='dense2')
        dropout3 = tf.layers.dropout(dense2, rate=DROPOUT, training=(mode == tf.estimator.ModeKeys.TRAIN), name='dropout3')

            ## Note: Replace hardcoding in next line with shape of pool2 (minus the batch size)
    #         pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 128])
    #         dense = tf.layers.dense(inputs = pool2_flat, units = 1024, activation = tf.nn.relu)
    #         dropout = tf.layers.dropout(inputs = dense, rate = DROPOUT, training =(mode == tf.estimator.ModeKeys.TRAIN) )
        logits = tf.layers.dense(inputs = dropout3, units = 10)
        predictions = { "classes": tf.argmax(input = logits, axis = 1),
                            "probabilities": tf.nn.softmax(logits, name = "smt")
                          }
        onehot_labels = tf.one_hot( indices = tf.cast(labels, tf.int32), depth=10 )
        loss = tf.losses.softmax_cross_entropy( onehot_labels= onehot_labels, logits= logits)
        tf.summary.scalar('loss', loss)
        
#         summary_hook = tf.train.SummarySaverHook(
#             100,
#             output_dir='tf_logs/summary8',
#             summary_op=tf.summary.merge_all())
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate= LEARNING_RATE)
            train_op = optimizer.minimize(loss = loss, global_step = tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)
        if mode == tf.estimator.ModeKeys.EVAL:
            eval_metric_ops = {"accuracy": tf.metrics.accuracy( labels = labels, predictions = predictions["classes"])}
            return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops = eval_metric_ops)
        else:
            eval_metric_ops = {"accuracy": tf.metrics.accuracy( labels = labels, predictions = predictions["classes"])}
            return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops = eval_metric_ops)


In [6]:
# def main(unused_argv):

#Set hyperparameters
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.3

# mnist = tf.contrib.learn.datasets.load_dataset("mnist")

original_train_data = X_train.T
original_train_labels = np.array(y_train, dtype = np.int32)
# train_data = original_train_data[0:int((original_train_data.shape[0] * 0.8))]
# train_labels = original_train_labels[0:int((original_train_data.shape[0] * 0.8))]

# eval_data = original_train_data[int((original_train_data.shape[0] * 0.8)):int(original_train_data.shape[0])]
# eval_labels =  original_train_labels[int((original_train_data.shape[0] * 0.8)):int(original_train_data.shape[0])]

train_data = original_train_data
train_labels = y_train
eval_data = X_validation.T
eval_labels = y_validation

print("Train data shape: ", train_data.shape)
print("Train labels shape: ", train_labels.shape)
print("Evals data shape: ", eval_data.shape)
print("Evals labels shape: ", eval_labels.shape)

#estimator
mnist_classifier = tf.estimator.Estimator(model_fn = cnn_model)

# summary_hook = tf.train.SummarySaverHook(
#             100,
#             output_dir='tf_logs/summary8',
#             summary_op=tf.summary.merge_all())

#log
tensor_to_log = {"probabilities": "smt"}
logging_hook = tf.train.LoggingTensorHook(tensors = tensor_to_log, every_n_iter = 50)

train_input_fn = tf.estimator.inputs.numpy_input_fn( x = {"x": train_data}, y = train_labels, batch_size = 128, num_epochs = None, shuffle = True)
mnist_classifier.train(input_fn = train_input_fn, steps = 20000, hooks = [])

Train data shape:  (45000, 3072)
Train labels shape:  (45000,)
Evals data shape:  (5000, 3072)
Evals labels shape:  (5000,)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpf2dzcudy', '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpf2dzcudy/model.ckpt.
INFO:tensorflow:loss = 2.31619, step = 1


KeyboardInterrupt: 

In [6]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x = {"x": eval_data}, y = eval_labels, num_epochs = 1, shuffle = False)
eval_results = mnist_classifier.evaluate(input_fn = eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2017-10-26-19:00:29
INFO:tensorflow:Restoring parameters from /tmp/tmp_jq0ctmu/model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2017-10-26-19:00:30
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.7928, global_step = 20000, loss = 0.854671
{'accuracy': 0.79280001, 'loss': 0.85467118, 'global_step': 20000}
